## Hyperparameter Tuning menggunakan GridSearch dan Optuna

## LIBRARY

In [1]:
#!pip install Sastrawi

# Standard library for data sciene
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Library for text processing
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer

# Library for NLP
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Library for basic machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator

# Library for deep learning
import tensorflow as tf
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

## LOAD DATA

In [2]:
df = pd.read_csv("D:/RIZQATSY/RIZQA/kuliah/DEEP LEARNING PYTHON/DEEP LEARNING HYPERPARAMETER/dataset/movie_review.CSV")
df.head()
df.shape

(200, 3)

In [3]:
df.head()

,Id,Sentiment,Text Tweet
0,1,negative,Jelek filmnya... apalagi si ernest gak mutu bg...
1,2,negative,Film king Arthur ini film paling jelek dari se...
2,3,negative,@beexkuanlin Sepanjang film gwa berkata kasar ...
3,4,negative,Ane ga suka fast and furious..menurutku kok je...
4,5,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id          200 non-null    int64 
 1   Sentiment   200 non-null    object
 2   Text Tweet  200 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


Interpretasi :

Data yang digunakan adalah ulasan film yang ada pada twitter. Dimana dataset tweet ini bersumber dari kaggle, yang dapat diunduh pada link berikut https://raw.githubusercontent.com/rijalghodi/datasets/main/movie_review.csv . Dataset terdiri dari 200 entri tanpa entri null. Setiap baris terdiri dari kolom id, sentimen, dan text tweet. Masing-masing kolom memiliiki tipe data yang sudah sesuai
<br>
fitur-fitur yang ada pada dataset tersebut yaitu
- id: identifier unik setiap tweet
- Sentiment: Label sentimen (negative, positive, neutral) atas film
- Text Tweet: Teks tweet yang mengulas film

<br>
Dari dataset tersebut akan dilakuka analisis sentimen menggunakan LSTM dan Hyperparameter Tuning GridSearch dan Optuna. Dengan tujuan dari analisis ini adalah untuk membangun model dengan akurasi terbaik untuk memprediksi sentimen yang terkandung pada tweet ulasan film

## Text Preprocessing

### 1. Cleaning text

In [5]:
def cleaning_text(text):
    # remove url
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)

    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)

    # remove mention handle user (@)
    text = re.sub(r'@[\w]*', ' ', text)

    # remove emojis
    emoji_pattern = re.compile(
        '['
        '\U0001F600-\U0001F64F'  # emoticons
        '\U0001F300-\U0001F5FF'  # symbols & pictographs
        '\U0001F680-\U0001F6FF'  # transport & map symbols
        '\U0001F700-\U0001F77F'  # alchemical symbols
        '\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
        '\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
        '\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
        '\U0001FA00-\U0001FA6F'  # Chess Symbols
        '\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
        '\U00002702-\U000027B0'  # Dingbats
        '\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # remove punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in text.lower():
        if x in punctuations:
            text = text.replace(x, " ")

    # remove extra whitespace
    text = ' '.join(text.split())

    # lowercase
    text = text.lower()
    return text

Interpretasi : <br>
Pada tahap pertama ini yaitu menghilangkan teks-teks yang tidak relevan, seperti link, tagar, tanda baca, emoji, dan mengubah semua huruf menjadi huruf kecil


### 2. Stopwords

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import requests

# CONSTRUCT STOPWORDS
rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
nltk_stopword = stopwords.words('indonesian')

# create path url for each stopword
path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword,
                  aliakbar_stopword, pebahasa_stopword, elang_stopword]

# combine stopwords
stopwords_l = nltk_stopword
for path in path_stopwords:
    response = requests.get(path)
    stopwords_l += response.text.split('\n')

custom_st = '''
yg yang dgn ane smpai bgt gua gwa si tu ama utk udh btw
ntar lol ttg emg aj aja tll sy sih kalo nya trsa mnrt nih
ma dr ajaa tp akan bs bikin kta pas pdahl bnyak guys abis tnx
bang banget nang mas amat bangettt tjoy hemm haha sllu hrs lanjut
bgtu sbnrnya trjadi bgtu pdhl sm plg skrg
'''

# create dictionary with unique stopword
st_words = set(stopwords_l)
custom_stopword = set(custom_st.split())

# result stopwords
stop_words = st_words | custom_stopword
print(f'Stopwords: {list(stop_words)[:5]}')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rizqatasyaa.zahra\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rizqatasyaa.zahra\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!


Stopwords: ['terlebih', 'udh', 'berkata', 'dimintai', 'pagi']


In [7]:
# remove stopwords
from nltk import word_tokenize, sent_tokenize

def remove_stopword(text, stop_words=stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

Interpretasi : <br>
Tahap selanjunya adalah stopwords, menghilangkan kata-kata yang umumnya diabaikan dalam proses analisis teks karena kurangnya informasi atau makna yang signifikan. Contohnya, dalam bahasa Indonesia, kata-kata seperti "dan", "atau", "yang", dan "di". Untuk menghasilkan model yang efisien dan akurat, kita harus menghilangkan stopword.

### 3. Stemming / Lemmatization

In [8]:
# stemming and lemmatization
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming_and_lemmatization(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)

Interpretasi : <br>
- Stemming adalah proses normalisasi kata dengan menghapus imbuhan (affixes) dari kata untuk menghasilkan bentuk dasar, yang disebut lemma. Contohnya, kata "berlari" dan "menemukan" akan disederhanakan menjadi "lari", "temu".
- Lemmatization juga merupakan proses normalisasi kata-kata, tetapi lebih kompleks daripada stemming karena melibatkan analisis morfologi kata-kata untuk mengembalikan kata-kata ke bentuk dasarnya, yang disebut "lemma". Contohnya, kata "mempertanggung-jawabkan" akan disederahanakan menjadi "tanggung-jawab". <br
>
Keduanya seringkali menghasilkan kata dasar yang sama tapi dalam beberapa kasus menghasilkan perbedaan.

### 4. Tokenization

In [9]:
# tokenization
def tokenize(text):
    return word_tokenize(text)

Interpretasi : <br>
Proses memecah teks atau korpus menjadi unit-unit yang lebih kecil, yang disebut dengan token. Token-token ini bisa berupa kata, frasa, simbol, atau unsur-unsur lain yang memiliki makna, tergantung pada konteks dan kebutuhan dari tugas yang sedang dilakukan.

Tokenisasi merupakan langkah penting dalam pemrosesan bahasa alami (NLP) dan analisis teks, karena memungkinkan komputer untuk memahami dan memproses data teks secara lebih efektif.

### 5. Gabungkan Semua Step Pre-Processing

In [10]:
# example
text = 'Jelek filmnya... apalagi si ernest gak mutu bgt actingnya... film sampah 💩 #kecewa https://x.com/alnurulg/status/1761301921846140991?s=20'
print(f'Original text: \n{text}\n')

# cleaning text and lowercase
text = cleaning_text(text)
print(f'Cleaned text: \n{text}\n')

# remove stopwords
text = remove_stopword(text)
print(f'Removed stopword: \n{text}\n')

# stemming and lemmatization
text = stemming_and_lemmatization(text)
print(f'Stemmed and lemmatized: \n{text}\n')

# tokenization
text = tokenize(text)
print(f'Tokenized: \n{text}')

Original text: 
Jelek filmnya... apalagi si ernest gak mutu bgt actingnya... film sampah 💩 #kecewa https://x.com/alnurulg/status/1761301921846140991?s=20

Cleaned text: 
jelek filmnya apalagi si ernest gak mutu bgt actingnya film sampah kecewa

Removed stopword: 
jelek filmnya ernest gak mutu actingnya film sampah kecewa

Stemmed and lemmatized: 
jelek film ernest gak mutu actingnya film sampah kecewa

Tokenized: 
['jelek', 'film', 'ernest', 'gak', 'mutu', 'actingnya', 'film', 'sampah', 'kecewa']


### 6. Menerapkan ke Dataset

In [11]:
# pipeline preprocess
def preprocess(text):
    # cleaning text and lowercase
    output = cleaning_text(text)

    # remove stopwords
    output = remove_stopword(output)

    # stemming and lemmatization
    output = stemming_and_lemmatization(output)

    # tokenization - Kita hapus dulu tokenisasi
    # output = tokenize(output)

    return output

In [12]:
# implement preprocessing
preprocessed_df = df.copy()

In [13]:
# Apply preprocessing to 'tweet' column
preprocessed_df['processed_tweet'] = df['Text Tweet'].apply(preprocess)

In [14]:
preprocessed_df

,Id,Sentiment,Text Tweet,processed_tweet
0,1,negative,Jelek filmnya... apalagi si ernest gak mutu bg...,jelek film ernest gak mutu actingnya film sampah
1,2,negative,Film king Arthur ini film paling jelek dari se...,film king arthur film jelek cerita king arthur
2,3,negative,@beexkuanlin Sepanjang film gwa berkata kasar ...,film kasar bapak
3,4,negative,Ane ga suka fast and furious..menurutku kok je...,ga suka fast and furious turut jelek film
4,5,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang...",ga tau film bilang perang perang perang an rp ...
...,...,...,...,...
195,196,positive,Fargo juga adaptasi dari film yang cukup berha...,fargo adaptasi film hasil season 1 best
196,197,positive,637.000 waw ini sangat keren flm horor dng jum...,637 000 waw keren flm horor dng tonton gin dlm 4
197,198,positive,@filmziarah film yang tenang dan menghanyutkan...,film tenang hanyut salut mbah ponco bawa karak...
198,199,positive,Film yg amat menarik. Kisah cinta & kesetiaan ...,film tarik kisah cinta setia saji kgn nenek re...


Interpretasi : <br>
Diatas adalah hasil akhir dari text preprocessing, dapat dilihat pada tabel diatas

## Membangun Model LSTM dengan Hyperparameter Tuning

### LSTM DEFAULT

Masalah yang dihadapi adalah klasifikasi teks. Oleh karena itu, kita akan menggunakan deep learning untuk menyelesaikannya.

Karena teks adalah data sekuen yang sangat sensitif terhadap urutan, kita akan menggunakan layer LSTM dalam deep learning untuk membangun model dengan akurasi yang tinggi.

Untuk model dasar ini, kita akan menggunakan konfigurasi sebagai berikut:
- Penggunaan LabelEncoder untuk encoding label.
- Pembagian data dengan train_test_split menggunakan 80% data untuk pelatihan dan 20% untuk pengujian.
- Model LSTM dengan layer embedding, 100 unit LSTM, dan satu neuron output dengan aktivasi sigmoid.
- Fungsi loss binary_crossentropy, optimizer adam, dan metrik accuracy.
- Pelatihan model selama 50 epoch dengan ukuran batch 32, serta evaluasi menggunakan data validasi.

data yang akan dianalisis adalah data yang sudah dilakukan preprocessing

In [15]:
# Tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_df['processed_tweet'])
X = tokenizer.texts_to_sequences(preprocessed_df['processed_tweet'])
X = pad_sequences(X, maxlen=100)  

In [16]:
# Encoding labels
y = LabelEncoder().fit_transform(preprocessed_df['Sentiment'])

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=X.shape[1]))
model_lstm.add(LSTM(100))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training LSTM model
model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/50


c:\Users\rizqatasyaa.zahra\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.5290 - loss: 0.6912 - val_accuracy: 0.5500 - val_loss: 0.6812
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6569 - loss: 0.6768 - val_accuracy: 0.6250 - val_loss: 0.6692
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7849 - loss: 0.6465 - val_accuracy: 0.6500 - val_loss: 0.6388
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7973 - loss: 0.5947 - val_accuracy: 0.7750 - val_loss: 0.6098
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8892 - loss: 0.5254 - val_accuracy: 0.7250 - val_loss: 0.6307
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9240 - loss: 0.4670 - val_accuracy: 0.7500 - val_loss: 0.5917
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9540 - loss: 0.3714 - val_accuracy: 0.8000 - val_loss: 0.5215
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.9477 - loss: 0.2355 - val_accuracy: 0.7250 - val_loss: 0.6254
Epoch 9/50

In [18]:
# Evaluating LSTM model
_, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print(f"Accuracy LSTM setelah preprocessing: {accuracy_lstm:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7604 - loss: 0.5656
Accuracy LSTM setelah preprocessing: 0.7500


Interpretasi : <br>

- Akurasi model LSTM setelah preprocessing adalah 75.00%. Ini berarti bahwa model mampu mengklasifikasikan dengan benar sekitar 75% dari data pengujian setelah proses pelatihan.
- Nilai loss sebesar 0.5656 menunjukkan bahwa model memiliki beberapa kesalahan dalam prediksinya. Nilai loss yang lebih rendah umumnya lebih baik, tetapi yang lebih penting adalah melihat keseimbangan antara loss dan akurasi.
- Akurasi sebesar 75.00% menunjukkan bahwa model cukup baik dalam melakukan tugas klasifikasi sentimen, namun masih bisa ditingkatkan lagi, misalnya melalui tuning hyperparameter, penggunaan arsitektur model yang lebih kompleks, atau peningkatan kualitas data training.
<br>

Dengan akurasi sekitar 75%, model ini sudah cukup baik untuk tugas klasifikasi dasar, tetapi untuk aplikasi yang memerlukan akurasi yang lebih tinggi, mungkin perlu eksplorasi lebih lanjut dan peningkatan model.



## Hyperparameter Tuning

### GridSearch

In [27]:
from sklearn.base import BaseEstimator, ClassifierMixin

# Define LSTM model as a custom LSTM classifier
class LSTMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, embedding_dim=100, lstm_units=100, optimizer='adam'):
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.optimizer = optimizer
    
    def fit(self, X, y):
        self.model = Sequential()
        self.model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=self.embedding_dim, input_length=X.shape[1]))
        self.model.add(LSTM(units=self.lstm_units))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])
        self.model.fit(X, y, epochs=50, batch_size=32, verbose=0)
        return self
    
    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype(int)
    
    def score(self, X, y):
        _, accuracy = self.model.evaluate(X, y)
        return accuracy


In [28]:
# Create an instance of the custom LSTM classifier
lstm_clf = LSTMClassifier()

# Define hyperparameters to search
param_grid = {
    'embedding_dim': [64, 100, 128, 256],
    'lstm_units': [64, 100, 128, 256],
    'optimizer': ['adam', 'rmsprop']
}

# Perform grid search using GridSearchCV
grid_search = GridSearchCV(estimator=lstm_clf, param_grid=param_grid, cv=3, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


c:\Users\rizqatasyaa.zahra\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8947 - loss: 0.2593 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7991 - loss: 0.7552 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8117 - loss: 0.6193 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8576 - loss: 0.3294  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8117 - loss: 0.6613 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7887 - loss: 0.4844 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9298 - loss: 0.2487 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7384 - loss: 0.9161  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7427 - loss: 0.8652 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8823 - loss: 0.3419 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7991 - loss: 0.5043 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8096 - loss: 0.5310 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9174 - loss: 0.2158 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0

GridSearchCV(cv=3, estimator=LSTMClassifier(),
             param_grid={'embedding_dim': [64, 100, 128, 256],
                         'lstm_units': [64, 100, 128, 256],
                         'optimizer': ['adam', 'rmsprop']},
             verbose=1)

Interpretasi : <br>

Pada hyperparameter tuning yg dilakukan menggunakan gridsearch , berikut adalah paramater yang akan dituning : <br>
- embedding_dim: [64, 100, 128, 256]
- lstm_units: [64, 100, 128, 256]
- optimizer: ['adam', 'rmsprop']

<br>
Dengan menggunakan GridSearchCV melakukan pencarian grid dengan cross-validation (cv=3) untuk menemukan kombinasi hyperparameter terbaik. dan Menggunakan layer embedding dan LSTM, dengan output layer sigmoid. Serta Cross-validation (cv=3) digunakan untuk mengevaluasi setiap kombinasi hyperparameter.

In [29]:
# Summarize results
print("Best: %f using %s" % (grid_search.best_score_, grid_search.best_params_))

# Evaluate best model on test data
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print(f"Accuracy LSTM grid search: {test_accuracy:.4f}")

Best: 0.868390 using {'embedding_dim': 128, 'lstm_units': 100, 'optimizer': 'rmsprop'}
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.7937 - loss: 0.7723   
Accuracy LSTM grid search: 0.8000


Interpretasi : <br>

Hasil tuning parameter gridsearch menghasilkan parameter terbaik yaitu : <br>
- embedding_dim: 128
- lstm_units: 100
- optimizer: rmsprop
<br>

Dengan skor terbaik yang dicapai selama grid search adalah sekitar 86.84%. Nilai ini merupakan rata-rata akurasi dari cross-validation (cv=3). Dan ini menunjukkan bahwa kombinasi tersebut memberikan performa terbaik selama evaluasi cross-validation. 
<br>

Sedangkan hasil akhir akurasi pada data uji sebesar 80.00% pada data uji menunjukkan bahwa model LSTM yang di-tuning dengan kombinasi terbaik ini cukup baik dalam mengklasifikasikan sentimen pada data yang belum pernah dilihat sebelumnya. dengan nilai loss sebesar 0.7723 menunjukkan bahwa masih ada ruang untuk perbaikan dalam mengurangi kesalahan prediksi model.
<br>

Hasil ini menunjukkan bahwa proses tuning hyperparameter berhasil meningkatkan performa model LSTM untuk tugas klasifikasi sentimen, tetapi performa sebenarnya pada data uji sedikit lebih rendah daripada skor cross-validation.

### Optuna

In [49]:
import optuna

# Define function to create LSTM model
def create_lstm_model(embedding_dim=100, lstm_units=100, optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=embedding_dim, input_length=X.shape[1]))
    model.add(LSTM(units=lstm_units))
    model.add(Dense(1, activation='sigmoid'))
    
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model




In [51]:
# Define objective function for Optuna
def objective(trial):
    # Define hyperparameters to optimize
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 100, 128, 256])
    lstm_units = trial.suggest_categorical('lstm_units', [64, 100, 128, 256])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'rmsprop'])
    epochs = trial.suggest_categorical('epochs', [25, 30, 35])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

    # Build and compile model
    model = create_lstm_model(embedding_dim=embedding_dim, lstm_units=lstm_units, optimizer=optimizer, learning_rate=learning_rate)
    
    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Evaluate the model
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    
    return accuracy

In [52]:
# Perform hyperparameter optimization using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Get best hyperparameters
best_params = study.best_params
best_accuracy = study.best_value

print(f"Optuna - Best Accuracy: {best_accuracy:.4f}")
print(f"Optuna - Best Hyperparameters: {best_params}")

[I 2024-07-05 13:59:08,441] A new study created in memory with name: no-name-0200cc32-0f5e-4938-88ed-eb95319a4fa5
C:\Users\rizqatasyaa.zahra\AppData\Local\Temp\ipykernel_11508\1216745967.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[I 2024-07-05 13:59:29,957] Trial 0 finished with value: 0.699999988079071 and parameters: {'embedding_dim': 256, 'lstm_units': 256, 'learning_rate': 0.00016579509420840922, 'optimizer': 'rmsprop', 'epochs': 30, 'batch_size': 64}. Best is trial 0 with value: 0.699999988079071.
[I 2024-07-05 13:59:39,797] Trial 1 finished with value: 0.625 and parameters: {'embedding_dim': 256, 'lstm_units': 128, 'learning_rate': 0.00016946592413932726, 'optimizer': 'rmsprop', 'epochs': 25, 'batch_size': 128}. Best is trial 0 with value: 0.69

Optuna - Best Accuracy: 0.8500
Optuna - Best Hyperparameters: {'embedding_dim': 128, 'lstm_units': 256, 'learning_rate': 0.0005001119871881131, 'optimizer': 'adam', 'epochs': 25, 'batch_size': 128}


Interpretasi : <br>

Pada hyperparameter tuning optuna, berikut parameter yang akan dioptimalkan : <br>
- embedding_dim: [64, 100, 128, 256]
- lstm_units: [64, 100, 128, 256]
- learning_rate: log-uniform antara 1e-4 dan 1e-2
- optimizer: ['adam', 'rmsprop']
- epochs: [25, 30, 35]
- batch_size: [32, 64, 128]
<br>

Hasil tuning optuna diperoleh akurasi terbaik yang dicapai oleh model LSTM dengan kombinasi hyperparameter yang dioptimalkan sebesar 0.8500, ini berarti model terbaik memiliki akurasi 85.00%. Dengan kombinasi parameter terbaik yg diperoleh adalah : <br>

- Embedding Dimension: 128
- LSTM Units: 256
- Learning Rate: 0.0005001119871881131
- Optimizer: Adam
- Epochs: 25
- Batch Size: 128
<br>

Optuna berhasil menemukan kombinasi hyperparameter yang optimal, meningkatkan performa model dibandingkan dengan penggunaan hyperparameter default 

In [61]:
# Rebuild and train the model with best hyperparameters
best_model_optuna = create_lstm_model(
    embedding_dim=best_params['embedding_dim'], 
    lstm_units=best_params['lstm_units'], 
    optimizer=best_params['optimizer'], 
    learning_rate=best_params['learning_rate']
)

best_model_optuna.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)
_, accuracy_optuna = best_model_optuna.evaluate(X_test, y_test, verbose=0)
print(f"Optuna - Final Test Accuracy: {accuracy_optuna:.4f}")

Epoch 1/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 119ms/step - accuracy: 0.4286 - loss: 0.6935
Epoch 2/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.5932 - loss: 0.6890
Epoch 3/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.5453 - loss: 0.6843
Epoch 4/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.5557 - loss: 0.6786
Epoch 5/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.5474 - loss: 0.6711
Epoch 6/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.5938 - loss: 0.6628
Epoch 7/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.7646 - loss: 0.6490
Epoch 8/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.7948 - loss: 0.6360
Epoch 9/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.8083 - loss: 0.6173
Epoch 10/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.8083 - loss: 0.5903
Epoch 11/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.7698 - loss: 0.5676
Epoch 12/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.7677 - lo

Interpretasi : <br>

Kemudian membangun kembali dan melatih model LSTM dengan hyperparameter terbaik yang ditemukan oleh Optuna, dan diperoleh akurasi akhir sebesar 0,8250 atau 82,50%. Akurasi ini menunjukkan bahwa model LSTM, setelah melalui proses tuning hyperparameter menggunakan Optuna, mampu menghasilkan prediksi yang benar sebesar 82.50% dari data uji. Ini berarti dari seluruh data uji yang digunakan untuk evaluasi, sekitar 82.50% diantaranya diklasifikasikan dengan benar berdasarkan sentimen yang dihasilkan oleh model.

## Kesimpulan

In [64]:
print(f"Default LSTM - Accuracy: {accuracy_lstm:.4f}")
print(f"GridSearchCV - Test Accuracy: {test_accuracy:.4f}")
print(f"Optuna - Final Test Accuracy: {accuracy_optuna:.4f}")

Default LSTM - Accuracy: 0.7500
GridSearchCV - Test Accuracy: 0.8000
Optuna - Final Test Accuracy: 0.8250


Interpretasi : <br>

- Proses optimasi hyperparameter menggunakan Optuna menghasilkan peningkatan signifikan dalam performa model LSTM dibandingkan dengan menggunakan hyperparameter default atau GridSearchCV.
- Meskipun GridSearchCV memberikan hasil yang lebih baik dari default, Optuna mampu menemukan kombinasi hyperparameter yang lebih baik lagi, mencapai akurasi tertinggi di antara ketiga pendekatan ini.
- Pemilihan metode optimasi hyperparameter seperti Optuna dapat menghemat waktu dan sumber daya komputasi, sambil meningkatkan performa model dengan cara yang lebih efisien.